## Imports

In [9]:
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk #pip install nltk
from nltk.corpus import stopwords
#nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ben/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [43]:
train = pd.read_csv('train_original.csv')

In [44]:
train.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


## Keyword

In [45]:
train['keyword'] = train['keyword'].str.replace('%20', ' ')


## Location

In [46]:
def clean_location(location):
    if location is not np.nan:
        location = location.lower()
        location = re.sub("[^a-zñ ]", "", location)
    
    return location

In [47]:
train['location'] = train['location'].replace('M!$$!$$!PP!', 'Mississippi')
train['location'] = train['location'].apply(clean_location)

In [48]:
train['location'].unique()[:1000:50]

array([nan, 'hagerstown md', 'amsterdam', 'the shire', 'milwaukee wi',
       'atx', 'arkansas jonesboro', 'az', 'me mammys belly',
       'under the blanket', 'united kingdom',
       'warwick ri dollarocracy also', 'sweden', 'santiago bernabeau',
       'el dorado ks', 'harris county texas', 'live ms',
       'lansing michigan', 'i love the smurfs ', 'miami'], dtype=object)

## Text

In [49]:
english = 'abcdefghijklmnopqrstuvwxyz0123456789 '
text = {}
len_english = len(english)
len_train = len(train['text'])
for i in range(0, len_train):
    len_text = len(train['text'].iloc[i])
    for j in range(0, len_text):
        lower_case = train['text'].iloc[i].lower()
        special_character = []
        if lower_case[j] not in english:
            if lower_case[j] in text:
                special_character = text[lower_case[j]]
            special_character.append(i)
            text[lower_case[j]] = special_character

In [50]:
text.keys()

dict_keys(['#', '.', "'", ',', '=', '>', '-', '?', '!', ';', ')', ':', '@', '/', '\x89', 'û', 'ò', 'ó', '\n', 'å', 'ê', '_', '&', '(', '[', ']', '|', '*', 'ã', '¢', '$', 'ï', '\x9d', 'ª', '+', '÷', 'ì', 'ñ', '%', '~', '£', '¤', '}', 'ü', '´', 'ç', 'è', 'â', '^', '¨', '©', '«', '\\', '¬', '¼', '¡', '`', '{'])

In [51]:
def show_sample():
    for key in text.keys():
        sample = text[key][0]
        print(train['text'].iloc[sample])
        
show_sample()

Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
Forest fire near La Ronge Sask. Canada
All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected
13,000 people receive #wildfires evacuation orders in California 
#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires
#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires
#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires
What's up man?
What a goooooooaaaaaal!!!!!!
London is cool ;)
London is cool ;)
Cooool :)
@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C
@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C
Barbados #Bridgetown JAMAICA ÛÒ Two cars set ablaze: SANTA CRUZ ÛÓ Head of the St Elizabeth Police Superintende...  http://t.co/wDUEaj8Q4J
Barbados #Bridg

In [52]:
def replace_text(text):
    text = text.replace('&amp', 'and')
    text = text.replace('amp;', 'and')
    text = text.replace('~', ' ')
    text = text.replace('Ûª', '\'')
    text = text.replace('ÛÒ', ' ')
    text = text.replace('ÛÓ', ' ')
    text = text.replace('&gt;', ' ')
    text = text.replace('&lt;', ' ')
    text = text.replace('ÛÏ', ' ')
    text = text.replace('Û', ' ')
    text = text.replace('\n', ' ')
    text = text.replace('å«', '\'')
    text = text.replace('åÊ', ' ')
    text = text.replace('åÇ', ' ')
    text = text.replace('Ì©', 'e')
    text = text.replace('Ì¤', 'c')
    text = text.replace('Ì¼', 'u')
    text = text.replace('Ì_', 'o')
    text = text.replace('`', '\'' )
    text = text.replace('\x89', '')
    text = text.replace('\x9d', '')
    
    return text

In [53]:
train['text'] = train['text'].apply(replace_text)

#### Buscamos palabras/signos que hayan quedado

In [63]:
porter=nltk.PorterStemmer()
one_letter="adhijklqruvwxyz"
double_letters="bcefgmnopst"

def clean_text(text):
    text = text.lower()
    
    text = re.sub("(http|https)\S+", "", text)
    text = re.sub(" can(\'*)t ", " cannot ", text)
    text = re.sub("n\'t ", " not ", text)
    text = re.sub("\'s ", " is ", text)
    text = re.sub("\'re ", " are ", text)
    text = re.sub("\'ve ", " have ", text)
    text = re.sub("\'d ", " would ", text)
    text = re.sub("\'m ", " am ", text)
    text = re.sub("\.|/|\+|-|÷|\*|:", " ", text)
    text = re.sub("( ' )|( # )", " ", text)
    text = re.sub("[^a-z 0-9]", "", text)
    
    
    for letter in one_letter:
        re_gex = letter + "(" + letter + "+)"
        text = re.sub(re_gex, letter, text)
        
    #Generalizamos para todas las palabras que contengan más de 2 letras iguales
    for letter in double_letters:
        double = letter + letter
        re_gex = double + "+"
        text = re.sub(re_gex, double, text)
    
    split_text = re.split("(\d+)", text)
    text = " ".join(split_text)
        
    text = re.sub("[0-9]\d*", "number", text)
    
    token_text = nltk.word_tokenize(text)    
    new_text = []
    for word in token_text:
        if word not in stopwords.words('english'):
            new_text.append(porter.stem(word))
            new_text.append(" ")
    
    text = "".join(new_text)
    text = re.sub(" +", " ", text)
    
    return text

In [66]:
train['text'] = train['text'].apply(clean_text)

In [67]:
show_sample()

deed reason thi earthquak may alah forgiv us al 
forest fire near la rong sask canada 
al resid ask shelter place notifi offic evacu shelter place order expect 
number peopl receiv wildfir evacu order california 
rockyfir updat california hwi number close direct due lake counti fire cafir wildfir 
rockyfir updat california hwi number close direct due lake counti fire cafir wildfir 
rockyfir updat california hwi number close direct due lake counti fire cafir wildfir 
man 
gooal 
london cool 
london cool 
cool 
bbcmtd wholes market ablaz 
bbcmtd wholes market ablaz 
barbado bridgetown jamaica two car set ablaz santa cruz head st elizabeth polic superintend 
barbado bridgetown jamaica two car set ablaz santa cruz head st elizabeth polic superintend 
barbado bridgetown jamaica two car set ablaz santa cruz head st elizabeth polic superintend 
barbado bridgetown jamaica two car set ablaz santa cruz head st elizabeth polic superintend 
outsid ablaz aliv dead insid 
polic arsonist delib set bl

In [68]:
tf_idf_vec = TfidfVectorizer(use_idf=True, smooth_idf=False, ngram_range=(1,1), stop_words='english', tokenizer=nltk.word_tokenize)
tf_idf_data = tf_idf_vec.fit_transform(train['text'])
col_names = tf_idf_vec.get_feature_names()

In [69]:
col_names

['ab',
 'aba',
 'abandon',
 'abandonedp',
 'abbandon',
 'abbott',
 'abbruchsimul',
 'abbswinston',
 'abbyairshow',
 'abc',
 'abcnew',
 'abcnorio',
 'abe',
 'aberdeen',
 'aberdeenfanpag',
 'aberdeenfc',
 'aberystwyth',
 'abha',
 'abia',
 'abil',
 'abject',
 'abl',
 'ablaz',
 'ableg',
 'abninfvet',
 'aboard',
 'abomb',
 'abomin',
 'abort',
 'abov',
 'abq',
 'abrancabalero',
 'absenc',
 'absolut',
 'absolutsumya',
 'abstorm',
 'abstract',
 'absurd',
 'absurdli',
 'abu',
 'abubara',
 'abysmaljoin',
 'ac',
 'acaciapenn',
 'academia',
 'acarewornheart',
 'acc',
 'accept',
 'access',
 'accid',
 'accidentali',
 'accidentalpropheci',
 'accionempresa',
 'accompani',
 'accord',
 'accordingli',
 'account',
 'accu',
 'accuraci',
 'accustom',
 'acd',
 'acdelco',
 'ace',
 'acebab',
 'acebreakingnew',
 'acenewsdesk',
 'aceorg',
 'acess',
 'ach',
 'achedin',
 'achiev',
 'achimota',
 'acid',
 'acmilan',
 'acn',
 'acoust',
 'acousticmaloley',
 'acquiesc',
 'acquir',
 'acquisit',
 'acr',
 'acronym',
 'acr

# HACER TRAIN/TEST.COPY() CON 'TEXT' Y 'ID'

In [ ]:
encoded_text = {}
tf_idf_array = tf_idf_data.toarray()
len_tf_idf_array = len(tf_idf_array)
len_col_names = len(col_names)
for i in range(len_col_names):
    data = {}
    for j in range(len_tf_idf_array):
        if tf_idf_array[j][i] != 0:
            data[j] = tf_idf_array[j][i]
        
    encoded_text[col_names[i]] = data

In [119]:
len_text = len(train['text'])

for i in range(len_text):
    values = []
    for word in train['text'].iloc[i].split():
          if word in encoded_text:
            values.append(encoded_text[word][i])
    if values:
        text['sum'].iloc[i] = sum(values)
        text['min'].iloc[i] = min(values)
        text['max'].iloc[i] = max(values)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 10 µs


In [120]:
text[['sum', 'min', 'max']].head(20)

,sum,min,max
0,2.539967,0.206936,0.504012
1,2.387767,0.302012,0.520764
2,3.925010,0.153758,0.585124
3,2.515355,0.150085,0.574226
4,3.069700,0.179350,0.436824
5,3.196590,0.100400,0.415266
6,3.652190,0.211922,0.421539
7,1.412882,0.675766,0.737116
8,2.220835,0.395912,0.528860
9,1.966376,0.391599,0.639305


## Resultado

In [71]:
train.iloc[10:110:10]

,id,keyword,location,text,target
10,16,NaN,NaN,three peopl die heat wave far,1
20,31,NaN,NaN,thi ridicul,0
30,44,NaN,NaN,end,0
40,59,ablaze,live on webcam,check nsfw,0
50,73,ablaze,sheffield township ohio,deputi man shot befor brighton home set ablaz,1
60,85,ablaze,NaN,revel wmv video mean mac farewel ablaz wmv en ...,0
70,100,accident,uk,personalinjuri accid thi summer read advic see...,0
80,117,accident,NaN,mom get home fast wish whi mom wa accid truck ...,0
90,132,accident,NaN,wa accid,0
100,144,accident,uk,norwaymfa bahrain polic previou die road accid...,1


## Guardado del dataframe

In [110]:
train.to_csv('train.csv', index=False)

In [122]:
text[['sum', 'max', 'min']].to_csv('train_text.csv', index=False)